In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}.")

output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'Ronaldo is known for his incredible work ethic and dedication to training. He reportedly follows a strict diet, works out daily, and even has a personal chef who travels with him to ensure he is eating the right foods.'

* Use of `.bind()`: To add arguments to a Runnable in a LCEL Chain.

For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [6]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'Cristiano '

## Combining LCEL Chains

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a sentence about {politician}.")

chain = prompt | model | StrOutputParser()

chain.invoke("Chamberlain")

'Chamberlain was a British politician who famously pursued a policy of appeasement towards Nazi Germany in the lead up to World War II.'

In [9]:
# combined chain
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

composed_chain.invoke({"politician": "Lincoln"})

'Yes, Abraham Lincoln is widely regarded as one of the greatest presidents in American history and his leadership during the Civil War and efforts to abolish slavery have had a positive impact on humanity. His dedication to preserving the Union and commitment to equality and justice for all individuals have left a lasting legacy that continues to inspire people around the world.'

In [ ]:
# another example: a chain inside another chain
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# defining manual prompts
prompt1 = ChatPromptTemplate.from_template("What is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in? Respond in {language}."
)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterand", "language": "French"})

'Le pays dont François Mitterrand était originaire, la France, se trouve en Europe.'

## LCEL Chain at work in a typical RAG App

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [13]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

# using prompt from the prompt hub
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

# parallel runnable
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/Users/abhinandansamal/anaconda3/envs/llmapp/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
/Users/abhinandansamal/anaconda3/envs/llmapp/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/abhinandansamal/anaconda3/envs/llmapp/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 

'Task Decomposition is a technique that involves breaking down complex tasks into smaller and simpler steps. This approach helps agents or models tackle difficult tasks more effectively by dividing them into manageable components. Different methods, such as Chain of Thought and Tree of Thoughts, can be used for task decomposition.'